In [2]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
!pip install transformers
from transformers import BertTokenizer, BertModel, BertForMaskedLM
from transformers import AdamW
import torch

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········


TIMEOUT: ignored

In [ ]:
def read_data(filepath):
  data = pd.read_csv(filepath)
  data = data[data["About"].isnull() == False]
  data = data[data["About"] != "0"]
  return data

data = read_data("/content/drive/My Drive/SDG_Research/SDG Compnies - rainmaking.io_impact_compass.csv")
data.head()


In [ ]:
data["Target"] = data["SDG Goal"].apply(lambda x: str(x) + "_goal")

labels = list(data["Target"].value_counts().keys().values)
print(labels)

print(data.shape)

In [ ]:
class SDGModel(torch.nn.Module):
  def __init__(self, output_size):
    super(SDGModel, self).__init__()
    self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    self.bert = BertModel.from_pretrained('bert-base-uncased')
    self.out = torch.nn.Linear(768, output_size)

  def forward(self, text):
    tokenized_text = self.tokenizer.tokenize("[CLS] " + text + " [SEP]")
    indexed_tokens = self.tokenizer.convert_tokens_to_ids(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])
    if tokens_tensor.shape[1] > 500:
      tokens_tensor = tokens_tensor[0][0:500].unsqueeze(0)
    outputs = self.bert(tokens_tensor)[0]
    return self.out(outputs[0][0])


In [ ]:
model = SDGModel(len(labels))

In [ ]:
model("Hello World")

tensor([ 0.6464, -0.1525,  0.3249,  0.0191,  0.2728, -0.0019],
       grad_fn=<AddBackward0>)

In [ ]:
optimizer = AdamW(model.parameters(), lr=1e-5)
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
n_epochs = 10

for e in range(1, n_epochs+1):
  current_loss = 0
  curr_data = data.sample(frac=1)
  for index, instance in curr_data.iterrows():
    optimizer.zero_grad()
    out = model(instance["About"].lower())
    loss = criterion(out.view(1, -1), torch.LongTensor([labels.index(instance["Target"])]))
    loss.backward()
    optimizer.step()
    current_loss += loss.item()
    if index % 10 == 0:
      print(f"Epoch={e}, index={index}, loss={current_loss/10}")
      print(f"Gold:{instance['Target']}, Predicted:{out}, Predicted_label:{labels[torch.argmax(out).item()]}")
      current_loss = 0



Epoch=1, index=40, loss=3.055637526512146
Gold:1_goal, Predicted:tensor([ 1.0000,  0.5410,  0.3340,  0.2435, -0.0698, -0.6557],
       grad_fn=<AddBackward0>), Predicted_label:3_goal
Epoch=1, index=500, loss=0.5805646181106567
Gold:3_goal, Predicted:tensor([ 1.0007,  0.5661,  0.3398,  0.2549, -0.0786, -0.6727],
       grad_fn=<AddBackward0>), Predicted_label:3_goal
Epoch=1, index=510, loss=1.7249140381813048
Gold:3_goal, Predicted:tensor([ 1.0021,  0.5938,  0.3442,  0.2559, -0.0887, -0.6838],
       grad_fn=<AddBackward0>), Predicted_label:3_goal
Epoch=1, index=30, loss=2.8420316100120546
Gold:1_goal, Predicted:tensor([ 1.0129,  0.5966,  0.3606,  0.2484, -0.0919, -0.7026],
       grad_fn=<AddBackward0>), Predicted_label:3_goal
Epoch=1, index=690, loss=4.862687492370606
Gold:5_goal, Predicted:tensor([ 0.9893,  0.7299,  0.3897,  0.2735, -0.1341, -0.7401],
       grad_fn=<AddBackward0>), Predicted_label:3_goal
Epoch=1, index=480, loss=0.8557445406913757
Gold:3_goal, Predicted:tensor([ 1.0